In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pygame
import random

# Initialize Pygame
pygame.init()

# Screen settings
WIDTH, HEIGHT = 1280, 720
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Fitness and Gesture Recognition Game')

# Fonts and Colors
FONT = pygame.font.Font(None, 36)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Initialize webcam
cap = cv2.VideoCapture(0)

# MediaPipe setup
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose()
hands = mp_hands.Hands()

# Function to draw text on the screen
def draw_text(text, x, y, color=WHITE):
    label = FONT.render(text, True, color)
    screen.blit(label, (x, y))

# Function to display the webcam feed in Pygame
def display_webcam_frame(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.rot90(img)
    img = pygame.surfarray.make_surface(img)
    screen.blit(img, (0, 0))

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle

# Push-up tracking function
def pushup_tracker():
    count = 0
    direction = 0
    form = 0
    feedback = "Fix Form"
    start_time = pygame.time.get_ticks()
    duration = 6000  # it is 60 seconds

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Calculate angles
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)

            # Check push-up form
            if elbow_angle > 160 and shoulder_angle > 40:
                form = 1
            
            # Count push-ups
            if form == 1:
                if elbow_angle <= 90 and direction == 0:
                    count += 1
                    direction = 1
                elif elbow_angle > 160 and direction == 1:
                    direction = 0

            feedback = "Perfect!" if form == 1 else "Fix Form"

            # Draw pose landmarks
            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Push-ups: {count}", 10, 10, GREEN)
        draw_text(feedback, 10, 50, GREEN if form == 1 else RED)

        # Check if time is up
        elapsed_time = pygame.time.get_ticks() - start_time
        remaining_time = max(0, (duration - elapsed_time) // 1000)
        draw_text(f"Time: {remaining_time}s", WIDTH - 150, 10, WHITE)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return count

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return count

# Hand gesture recognition function
def gesture_recognition():
    gestures = ["Fist", "Open Hand", "Peace Sign", "Thumbs Up"]
    target_gesture = random.choice(gestures)
    score = 0
    start_time = pygame.time.get_ticks()
    duration = 6000  # 6 seconds

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Simple gesture recognition logic
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

                if all(finger.y > thumb_tip.y for finger in [index_tip, middle_tip, ring_tip, pinky_tip]):
                    detected_gesture = "Fist"
                elif all(finger.y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y for finger in [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip]):
                    detected_gesture = "Open Hand"
                elif index_tip.y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y and middle_tip.y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y:
                    detected_gesture = "Peace Sign"
                elif thumb_tip.y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y and all(finger.y > thumb_tip.y for finger in [index_tip, middle_tip, ring_tip, pinky_tip]):
                    detected_gesture = "Thumbs Up"
                else:
                    detected_gesture = "Unknown"

                if detected_gesture == target_gesture:
                    score += 1
                    target_gesture = random.choice(gestures)

        display_webcam_frame(img)
        draw_text(f"Target Gesture: {target_gesture}", 10, 10, GREEN)
        draw_text(f"Score: {score}", 10, 50, GREEN)

        # Check if time is up
        elapsed_time = pygame.time.get_ticks() - start_time
        remaining_time = max(0, (duration - elapsed_time) // 1000)
        draw_text(f"Time: {remaining_time}s", WIDTH - 150, 10, WHITE)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return score

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return score

# Pose matching function
def pose_matching():
    poses = ["T-Pose", "Warrior Pose", "Tree Pose"]
    target_pose = random.choice(poses)
    score = 0
    start_time = pygame.time.get_ticks()
    duration = 6000  # 60 seconds

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Simple pose matching logic
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
            hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

            if 80 < shoulder_angle < 100 and hip_angle > 160 and knee_angle > 160:
                detected_pose = "T-Pose"
            elif 100 < shoulder_angle < 120 and 120 < hip_angle < 150 and 150 < knee_angle < 180:
                detected_pose = "Warrior Pose"
            elif hip_angle > 160 and 30 < knee_angle < 60:
                detected_pose = "Tree Pose"
            else:
                detected_pose = "Unknown"

            if detected_pose == target_pose:
                score += 1
                target_pose = random.choice(poses)

        display_webcam_frame(img)
        draw_text(f"Target Pose: {target_pose}", 10, 10, GREEN)
        draw_text(f"Score: {score}", 10, 50, GREEN)

        # Check if time is up
        elapsed_time = pygame.time.get_ticks() - start_time
        remaining_time = max(0, (duration - elapsed_time) // 1000)
        draw_text(f"Time: {remaining_time}s", WIDTH - 150, 10, WHITE)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return score

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return score

# Main game loop
def main_menu():
    pushup_score = 0
    gesture_score = 0
    pose_score = 0

    while True:
        screen.fill(BLACK)
        draw_text("Fitness and Gesture Recognition Game", WIDTH // 2 - 200, 100)
        draw_text("1. Push-up Challenge", WIDTH // 2 - 100, 200)
        draw_text("2. Gesture Recognition Challenge", WIDTH // 2 - 150, 250)
        draw_text("3. Pose Matching Challenge", WIDTH // 2 - 130, 300)
        draw_text("4. View Scores", WIDTH // 2 - 70, 350)
        draw_text("Q. Quit", WIDTH // 2 - 30, 400)

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_1:
                    pushup_score = pushup_tracker()
                elif event.key == pygame.K_2:
                    gesture_score = gesture_recognition()
                elif event.key == pygame.K_3:
                    pose_score = pose_matching()
                elif event.key == pygame.K_4:
                    screen.fill(BLACK)
                    draw_text(f"Push-up Score: {pushup_score}", WIDTH // 2 - 100, 200)
                    draw_text(f"Gesture Score: {gesture_score}", WIDTH // 2 - 100, 250)
                    draw_text(f"Pose Score: {pose_score}", WIDTH // 2 - 100, 300)
                    draw_text("Press any key to return", WIDTH // 2 - 120, 400)
                    pygame.display.update()
                    waiting = True
                    while waiting:
                        for event in pygame.event.get():
                            if event.type == pygame.KEYDOWN:
                                waiting = False
                elif event.key == pygame.K_q:
                    return

if __name__ == "__main__":
    main_menu()
    pygame.quit()
    cap.release()
    cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pygame
import random
import time

# Initialize Pygame
pygame.init()

# Screen settings
WIDTH, HEIGHT = 1280, 720
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Fitness and Gesture Recognition Game')

# Fonts and Colors
FONT = pygame.font.Font(None, 36)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Initialize webcam
cap = cv2.VideoCapture(0)

# MediaPipe setup
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Function to draw text on the screen
def draw_text(text, x, y, color=WHITE, size=36):
    font = pygame.font.Font(None, size)
    label = font.render(text, True, color)
    screen.blit(label, (x, y))

# Function to display the webcam feed in Pygame
def display_webcam_frame(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.rot90(img)
    img = pygame.surfarray.make_surface(img)
    screen.blit(img, (0, 0))

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle

# Function to create blurred timer background
def create_blurred_timer(time_left):
    surface = pygame.Surface((200, 50))
    surface.set_alpha(150)
    surface.fill((100, 100, 100))
    screen.blit(surface, (WIDTH - 220, 5))
    draw_text(f"Time: {time_left}s", WIDTH - 200, 10, WHITE)

# Improved push-up tracking function
def pushup_tracker():
    count = 0
    direction = 0
    form = 0
    feedback = "Get ready!"
    start_time = time.time()
    duration = 60  # 60 seconds

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Calculate angles
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)

            # Check push-up form
            if elbow_angle > 160 and shoulder_angle > 40:
                form = 1
            
            # Count push-ups
            if form == 1:
                if elbow_angle <= 90 and direction == 0:
                    count += 1
                    direction = 1
                    feedback = random.choice(["Great job!", "You're on fire!", "Keep it up!", "Awesome push-up!"])
                elif elbow_angle > 160 and direction == 1:
                    direction = 0

            feedback = feedback if form == 1 else random.choice(["Lower your body more!", "Keep your back straight!", "You can do it!"])

            # Draw pose landmarks
            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Push-ups: {count}", 10, 10, GREEN)
        draw_text(feedback, 10, 50, GREEN if form == 1 else RED)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return count

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return count


def gesture_recognition():
    gestures = ["Fist", "Open Hand", "Peace Sign", "Thumbs Up"]
    target_gesture = random.choice(gestures)
    score = 0
    start_time = time.time()
    duration = 60  # 60 seconds

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
                wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]

                # Calculate distances for more accurate detection
                finger_distances = [
                    np.linalg.norm(np.array([finger.x, finger.y]) - np.array([wrist.x, wrist.y]))
                    for finger in [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip]
                ]

                if all(distance < 0.1 for distance in finger_distances[1:]):
                    detected_gesture = "Fist"
                elif all(distance > 0.15 for distance in finger_distances):
                    detected_gesture = "Open Hand"
                elif finger_distances[1] > 0.15 and finger_distances[2] > 0.15 and all(distance < 0.1 for distance in finger_distances[3:]):
                    detected_gesture = "Peace Sign"
                elif finger_distances[0] > 0.15 and all(distance < 0.1 for distance in finger_distances[1:]) and thumb_index_angle > 30:
                    detected_gesture = "Thumbs Up"
                else:
                    detected_gesture = "Unknown"

                if detected_gesture == target_gesture:
                    score += 1
                    target_gesture = random.choice(gestures)

        display_webcam_frame(img)
        draw_text(f"Target Gesture: {target_gesture}", 10, 10, GREEN)
        draw_text(f"Score: {score}", 10, 50, GREEN)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return score

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return score


def pose_matching():
    poses = ["T-Pose", "Warrior Pose", "Tree Pose"]
    target_pose = random.choice(poses)
    score = 0
    start_time = time.time()
    duration = 60  # 60 seconds

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Improved pose matching logic
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
            hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

            if 80 < shoulder_angle < 100 and hip_angle > 160 and knee_angle > 160:
                detected_pose = "T-Pose"
            elif 100 < shoulder_angle < 120 and 120 < hip_angle < 150 and 150 < knee_angle < 180:
                detected_pose = "Warrior Pose"
            elif hip_angle > 160 and 30 < knee_angle < 60:
                detected_pose = "Tree Pose"
            else:
                detected_pose = "Unknown"

            if detected_pose == target_pose:
                score += 1
                target_pose = random.choice(poses)

        display_webcam_frame(img)
        draw_text(f"Target Pose: {target_pose}", 10, 10, GREEN)
        draw_text(f"Score: {score}", 10, 50, GREEN)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return score

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return score


def jumping_jack_counter():
    count = 0
    state = "down"
    start_time = time.time()
    duration = 60  # 60 seconds

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            angle = calculate_angle(left_shoulder, left_hip, left_ankle)

            if angle > 160 and state == "down":
                state = "up"
                count += 1
            elif angle < 140 and state == "up":
                state = "down"

            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Jumping Jacks: {count}", 10, 10, GREEN)
        
        feedback = random.choice(["Jump higher!", "You're doing great!", "Feel the burn!", "Keep it up!"])
        draw_text(feedback, 10, 50, GREEN)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return count

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return count


        
def plank_hold_timer():
    start_time = time.time()
    duration = 60 
    plank_time = 0
    is_planking = False

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            angle = calculate_angle(left_shoulder, left_hip, left_ankle)

            if 160 < angle < 200:
                if not is_planking:
                    is_planking = True
                    plank_start_time = time.time()
                else:
                    plank_time = time.time() - plank_start_time
            else:
                is_planking = False

            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Plank Time: {plank_time:.1f}s", 10, 10, GREEN)
        
        feedback = random.choice(["Hold it!", "Stay strong!", "You've got this!", "Feel those abs!"])
        draw_text(feedback, 10, 50, GREEN if is_planking else RED)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return plank_time

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return plank_time


def squat_counter():
    count = 0
    state = "up"
    start_time = time.time()
    duration = 60  

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            angle = calculate_angle(left_hip, left_knee, left_ankle)

            if angle < 100 and state == "up":
                state = "down"
            elif angle > 160 and state == "down":
                state = "up"
                count += 1

            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Squats: {count}", 10, 10, GREEN)
        
        feedback = random.choice(["Go lower!", "Feel the burn!", "Squeeze those glutes!", "You're a squat master!"])
        draw_text(feedback, 10, 50, GREEN if state == "down" else WHITE)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return count

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return count

# Improved main menu function
def main_menu():
    pushup_score = 0
    gesture_score = 0
    pose_score = 0
    jumping_jack_score = 0
    plank_time = 0
    squat_score = 0

    while True:
        screen.fill(BLACK)
        draw_text("Improved Fitness and Gesture Recognition Game", WIDTH // 2 - 250, 50, WHITE, 48)
        draw_text("1. Push-up Challenge", WIDTH // 2 - 100, 150)
        draw_text("2. Gesture Recognition Challenge", WIDTH // 2 - 150, 200)
        draw_text("3. Pose Matching Challenge", WIDTH // 2 - 130, 250)
        draw_text("4. Jumping Jack Challenge", WIDTH // 2 - 130, 300)
        draw_text("5. Plank Hold Challenge", WIDTH // 2 - 110, 350)
        draw_text("6. Squat Challenge", WIDTH // 2 - 90, 400)
        draw_text("7. View Scores", WIDTH // 2 - 70, 450)
        draw_text("Q. Quit", WIDTH // 2 - 30, 500)

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_1:
                    pushup_score = pushup_tracker()
                elif event.key == pygame.K_2:
                    gesture_score = gesture_recognition()
                elif event.key == pygame.K_3:
                    pose_score = pose_matching()
                elif event.key == pygame.K_4:
                    jumping_jack_score = jumping_jack_counter()
                elif event.key == pygame.K_5:
                    plank_time = plank_hold_timer()
                elif event.key == pygame.K_6:
                    squat_score = squat_counter()
                elif event.key == pygame.K_7:
                    screen.fill(BLACK)
                    draw_text("Your Fitness Scores", WIDTH // 2 - 100, 50, WHITE, 48)
                    draw_text(f"Push-ups: {pushup_score}", WIDTH // 2 - 100, 150)
                    draw_text(f"Gestures: {gesture_score}", WIDTH // 2 - 100, 200)
                    draw_text(f"Poses: {pose_score}", WIDTH // 2 - 100, 250)
                    draw_text(f"Jumping Jacks: {jumping_jack_score}", WIDTH // 2 - 100, 300)
                    draw_text(f"Plank Hold: {plank_time:.1f}s", WIDTH // 2 - 100, 350)
                    draw_text(f"Squats: {squat_score}", WIDTH // 2 - 100, 400)
                    draw_text("Press any key to return", WIDTH // 2 - 120, 500)
                    pygame.display.update()
                    waiting = True
                    while waiting:
                        for event in pygame.event.get():
                            if event.type == pygame.KEYDOWN:
                                waiting = False
                elif event.key == pygame.K_q:
                    return

if __name__ == "__main__":
    main_menu()
    pygame.quit()
    cap.release()
    cv2.destroyAllWindows()

pygame 2.6.0 (SDL 2.28.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import pygame
import random
import time


pygame.init()


WIDTH, HEIGHT = 1280, 720
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Gesture-Controlled Fitness Game')


FONT = pygame.font.Font(None, 36)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)


cap = cv2.VideoCapture(0)


mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)


def draw_text(text, x, y, color=WHITE, size=36):
    font = pygame.font.Font(None, size)
    label = font.render(text, True, color)
    screen.blit(label, (x, y))


def display_webcam_frame(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.rot90(img)
    img = pygame.surfarray.make_surface(img)
    screen.blit(img, (0, 0))


def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle


def create_blurred_timer(time_left):
    surface = pygame.Surface((200, 50))
    surface.set_alpha(150)
    surface.fill((100, 100, 100))
    screen.blit(surface, (WIDTH - 220, 5))
    draw_text(f"Time: {time_left}s", WIDTH - 200, 10, WHITE)


def detect_gesture(hand_landmarks):
    # Get key points for each finger's joints
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    
    index_pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]
    middle_pip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP]
    ring_pip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP]
    pinky_pip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP]
    
    wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
    
    def is_finger_extended(tip, pip, wrist):
        return tip.y < pip.y and pip.y < wrist.y 
    
    def is_thumb_up(thumb_tip, wrist):
        return thumb_tip.x > wrist.x  

    if all(not is_finger_extended(tip, pip, wrist) for tip, pip in zip(
        [index_tip, middle_tip, ring_tip, pinky_tip],
        [index_pip, middle_pip, ring_pip, pinky_pip])):
        return "Fist"
    
    elif all(is_finger_extended(tip, pip, wrist) for tip, pip in zip(
        [index_tip, middle_tip, ring_tip, pinky_tip],
        [index_pip, middle_pip, ring_pip, pinky_pip])):
        return "Open Hand"
    
    elif is_finger_extended(index_tip, index_pip, wrist) and is_finger_extended(middle_tip, middle_pip, wrist) and \
         not is_finger_extended(ring_tip, ring_pip, wrist) and not is_finger_extended(pinky_tip, pinky_pip, wrist):
        return "Peace Sign"
    
    elif is_thumb_up(thumb_tip, wrist) and all(not is_finger_extended(tip, pip, wrist) for tip, pip in zip(
        [index_tip, middle_tip, ring_tip, pinky_tip],
        [index_pip, middle_pip, ring_pip, pinky_pip])):
        return "Thumbs Up"
    
    else:
        return "Unknown"




def main_menu():
    pushup_score = 0
    gesture_score = 0
    pose_score = 0
    jumping_jack_score = 0
    plank_time = 0
    squat_score = 0

    menu_options = [
        "Push-up Challenge",
        "Gesture Recognition Challenge",
        "Pose Matching Challenge",
        "Jumping Jack Challenge",
        "Plank Hold Challenge",
        "Squat Challenge",
        "View Scores",
        "Quit"
    ]
    selected_option = 0
    last_gesture_time = time.time()
    gesture_cooldown = 1.0

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        hand_results = hands.process(img_rgb)

        screen.fill(BLACK)
        display_webcam_frame(img)

        draw_text("Gesture-Controlled Fitness Game", WIDTH // 2 - 250, 50, WHITE, 48)

        for i, option in enumerate(menu_options):
            color = GREEN if i == selected_option else WHITE
            draw_text(f"{i + 1}. {option}", WIDTH // 2 - 150, 150 + i * 50, color)

        detected_gesture = "No hand detected"
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                detected_gesture = detect_gesture(hand_landmarks)

                current_time = time.time()
                if current_time - last_gesture_time > gesture_cooldown:
                    if detected_gesture == "Open Hand" and selected_option > 0:
                        selected_option -= 1
                        last_gesture_time = current_time
                    elif detected_gesture == "Fist" and selected_option < len(menu_options) - 1:
                        selected_option += 1
                        last_gesture_time = current_time
                    elif detected_gesture == "Peace Sign":
                        # Execute selected option
                        if selected_option == 0:
                            pushup_score = pushup_tracker()
                        elif selected_option == 1:
                            gesture_score = gesture_recognition()
                        elif selected_option == 2:
                            pose_score = pose_matching()
                        elif selected_option == 3:
                            jumping_jack_score = jumping_jack_counter()
                        elif selected_option == 4:
                            plank_time = plank_hold_timer()
                        elif selected_option == 5:
                            squat_score = squat_counter()
                        elif selected_option == 6:
                            view_scores(pushup_score, gesture_score, pose_score, jumping_jack_score, plank_time, squat_score)
                        elif selected_option == 7:
                            return
                        last_gesture_time = current_time

        # Debug information
        draw_text(f"Detected Gesture: {detected_gesture}", 10, HEIGHT - 60, BLUE)
        draw_text(f"Selected Option: {selected_option + 1}", 10, HEIGHT - 30, BLUE)

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return

def view_scores(pushup_score, gesture_score, pose_score, jumping_jack_score, plank_time, squat_score):
    waiting = True
    while waiting:
        screen.fill(BLACK)
        draw_text("Your Fitness Scores", WIDTH // 2 - 100, 50, WHITE, 48)
        draw_text(f"Push-ups: {pushup_score}", WIDTH // 2 - 100, 150)
        draw_text(f"Gestures: {gesture_score}", WIDTH // 2 - 100, 200)
        draw_text(f"Poses: {pose_score}", WIDTH // 2 - 100, 250)
        draw_text(f"Jumping Jacks: {jumping_jack_score}", WIDTH // 2 - 100, 300)
        draw_text(f"Plank Hold: {plank_time:.1f}s", WIDTH // 2 - 100, 350)
        draw_text(f"Squats: {squat_score}", WIDTH // 2 - 100, 400)
        draw_text("Make a Peace Sign to return", WIDTH // 2 - 120, 500)

        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        hand_results = hands.process(img_rgb)

        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                detected_gesture = detect_gesture(hand_landmarks)
                if detected_gesture == "Open Hand":
                    waiting = False

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return

def pushup_tracker():
    count = 0
    direction = 0
    form = 0
    feedback = "Get ready!"
    start_time = time.time()
    duration = 60

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Calculate angles
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)

            # Check push-up form
            if elbow_angle > 160 and shoulder_angle > 40:
                form = 1
            
            # Count push-ups
            if form == 1:
                if elbow_angle <= 90 and direction == 0:
                    count += 1
                    direction = 1
                    feedback = random.choice(["Great job!", "You're on fire!", "Keep it up!", "Awesome push-up!"])
                elif elbow_angle > 160 and direction == 1:
                    direction = 0

            feedback = feedback if form == 1 else random.choice(["Lower your body more!", "Keep your back straight!", "You can do it!"])

            # Draw pose landmarks
            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Push-ups: {count}", 10, 10, GREEN)
        draw_text(feedback, 10, 50, GREEN if form == 1 else RED)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return count

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return count


def gesture_recognition():
    gestures = ["Fist", "Open Hand", "Peace Sign"] #Thumbs up in progress not yet done
    target_gesture = random.choice(gestures)
    score = 0
    start_time = time.time()
    duration = 60  

    while True:
        ret, img = cap.read()  
        if not ret:
            print("Camera error")
            break
        
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)


                detected_gesture = detect_gesture(hand_landmarks)
                
                if detected_gesture == target_gesture:
                    score += 1
                    target_gesture = random.choice(gestures)

        display_webcam_frame(img)
        draw_text(f"Target Gesture: {target_gesture}", 10, 10, (0, 255, 0))
        draw_text(f"Score: {score}", 10, 50, (0, 255, 0))

        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, (255, 0, 0))
            pygame.display.update()
            pygame.time.wait(2000)
            return score

        pygame.display.update()

        if pygame.event.peek(pygame.QUIT):
            return score



def pose_matching():
    poses = ["T-Pose", "Warrior Pose", "Tree Pose"]
    target_pose = random.choice(poses)
    score = 0
    start_time = time.time()
    duration = 60 

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
            hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

            right_shoulder_angle = calculate_angle(right_elbow, right_shoulder, right_hip)
            right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

            if (80 < shoulder_angle < 100 and 80 < right_shoulder_angle < 100 and
                    hip_angle > 160 and right_hip_angle > 160 and
                    knee_angle > 160 and right_knee_angle > 160):
                detected_pose = "T-Pose"
            elif (100 < shoulder_angle < 120 and 120 < hip_angle < 150 and
                  150 < knee_angle < 180 and
                  100 < right_shoulder_angle < 120 and
                  120 < right_hip_angle < 150 and
                  150 < right_knee_angle < 180):
                detected_pose = "Warrior Pose"
            elif (hip_angle > 160 and 30 < knee_angle < 60 and
                  right_hip_angle > 160 and 30 < right_knee_angle < 60):
                detected_pose = "Tree Pose"
            else:
                detected_pose = "Unknown"

            if detected_pose == target_pose:
                score += 1
                target_pose = random.choice(poses)

        display_webcam_frame(img)
        draw_text(f"Target Pose: {target_pose}", 10, 10, GREEN)
        draw_text(f"Score: {score}", 10, 50, GREEN)

        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return score

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return score

        

def jumping_jack_counter():
    count = 0
    state = "down"
    start_time = time.time()
    duration = 60
    feedback_timer = start_time
    feedback = "Let's start!"

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            left_leg_angle = calculate_angle(left_shoulder, left_hip, left_ankle)
            right_leg_angle = calculate_angle(right_shoulder, right_hip, right_ankle)

            average_leg_angle = (left_leg_angle + right_leg_angle) / 2

            if average_leg_angle > 160 and state == "down":
                state = "up"
                count += 1
            elif average_leg_angle < 140 and state == "up":
                state = "down" 


            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Jumping Jacks: {count}", 10, 10, GREEN)

        current_time = time.time()
        if current_time - feedback_timer > 5:
            feedback = random.choice(["Jump higher!", "You're doing great!", "Feel the burn!", "Keep it up!"])
            feedback_timer = current_time  # Reset feedback timer

        draw_text(feedback, 10, 50, GREEN)

        elapsed_time = current_time - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return count

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return count



        
def plank_hold_timer():
    start_time = time.time()
    duration = 60 
    plank_time = 0
    is_planking = False

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            angle = calculate_angle(left_shoulder, left_hip, left_ankle)

            if 160 < angle < 200:
                if not is_planking:
                    is_planking = True
                    plank_start_time = time.time()
                else:
                    plank_time = time.time() - plank_start_time
            else:
                is_planking = False

            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Plank Time: {plank_time:.1f}s", 10, 10, GREEN)
        
        feedback = random.choice(["Hold it!", "Stay strong!", "You've got this!", "Feel those abs!"])
        draw_text(feedback, 10, 50, GREEN if is_planking else RED)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return plank_time

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return plank_time


def squat_counter():
    count = 0
    state = "up"
    start_time = time.time()
    duration = 60  

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            angle = calculate_angle(left_hip, left_knee, left_ankle)

            if angle < 100 and state == "up":
                state = "down"
            elif angle > 160 and state == "down":
                state = "up"
                count += 1

            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        display_webcam_frame(img)
        draw_text(f"Squats: {count}", 10, 10, GREEN)
        
        feedback = random.choice(["Go lower!", "Feel the burn!", "Squeeze those glutes!", "You're a squat master!"])
        draw_text(feedback, 10, 50, GREEN if state == "down" else WHITE)

        # Display blurred timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, int(duration - elapsed_time))
        create_blurred_timer(remaining_time)

        if elapsed_time >= duration:
            draw_text("Time's up!", WIDTH // 2 - 50, HEIGHT // 2, RED)
            pygame.display.update()
            pygame.time.wait(2000)
            return count

        pygame.display.update()
        if pygame.event.peek(pygame.QUIT):
            return count



if __name__ == "__main__":
    main_menu()
    pygame.quit()
    cap.release()
    cv2.destroyAllWindows()